# Producing Mercury Challenge MA Baserate Models

This notebook illustrates a backtest of the Mercury Challenge Baserate models for Military Activity.  See [Developing Mercury Time Series Models](./Fit%20CU%20Time%20Series%20Models.ipynb) for model development details.

In [1]:
%matplotlib inline

In [2]:
import os, re
import json
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import (plot_acf, plot_pacf)
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import datetime
from dateutil.parser import parse
import sys
sys.path.append(("../.."))
from Baserate.main.baserate import (
    MaBaserate
)
from ExpressScore.main.express_score import MaScorer
import warnings
warnings.filterwarnings("ignore")

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use("fivethirtyeight");

In the cell below replace the dates for `WARN_START_DATE` and `WARN_END_DATE`.

In [3]:
EVT_ABBR = "MA"
MC_HOME = os.path.abspath("../../..")
DOC_PATH = os.path.join(MC_HOME, "doc")
EVT_DOC_PATH = os.path.join(DOC_PATH, "scoring", EVT_ABBR)
DATA_PATH = os.path.join(MC_HOME, "data")
GSR_PATH = os.path.join(DATA_PATH, "gsr")
MA_GSR_PATH = os.path.join(GSR_PATH, "ma_gsr")
WARN_PATH = os.path.join(DATA_PATH, "baserate_warnings")
MA_WARN_PATH = os.path.join(WARN_PATH, "baserate_ma_warnings")
RESOURCE_PATH = os.path.join(MC_HOME, "src", "Baserate", "resources")
HISTORY_LOOKBACK = 365
HISTORY_DELAY = 15
WARN_START_DATE = "2018-01-09"
WARN_END_DATE = "2019-01-22"
test_range = pd.date_range(WARN_START_DATE, WARN_END_DATE)
test_months = set([dd.strftime("%B_%Y") for dd in test_range])
PARTICIPANT_ID = "Mercury_Baserate"

In Challenge Period 2 we are only going to score Syria and Iraq.

In [4]:
MA_COUNTRIES = ["Iraq", "Syria"]

In [5]:
test_months

{'April_2018',
 'August_2018',
 'December_2018',
 'February_2018',
 'January_2018',
 'January_2019',
 'July_2018',
 'June_2018',
 'March_2018',
 'May_2018',
 'November_2018',
 'October_2018',
 'September_2018'}

Load the GSR from the directory

In [6]:
gsr_files = [x for x in os.listdir(MA_GSR_PATH) if x.endswith("json")]
print(gsr_files)
gsr = []
for gf in gsr_files:
    with open(os.path.join(MA_GSR_PATH, gf), "r", encoding="utf-8-sig") as f:
        monthly_gsr = json.load(f)
        gsr += monthly_gsr
print(len(gsr))

['MA_May_2015.json', 'MA_August_2018.json', 'MA_September_2017.json', 'MA_January_2016.json', 'MA_November_2017.json', 'MA_October_2017.json', 'MA_April_2017.json', 'MA_March_2018.json', 'MA_August_2015.json', 'MA_February_2018.json', 'MA_May_2018.json', 'MA_April_2016.json', 'MA_October_2016.json', 'MA_November_2016.json', 'MA_January_2017.json', 'MA_September_2016.json', 'MA_June_2015.json', 'MA_July_2016.json', 'MA_December_2016.json', 'MA_June_2018.json', 'MA_December_2017.json', 'MA_July_2017.json', 'MA_June_2017.json', 'MA_December_2018.json', 'MA_July_2018.json', 'MA_July_2015.json', 'MA_June_2016.json', 'MA_December_2015.json', 'MA_October_2015.json', 'MA_November_2015.json', 'MA_September_2015.json', 'MA_March_2017.json', 'MA_February_2017.json', 'MA_May_2017.json', 'MA_January_2018.json', 'MA_August_2016.json', 'MA_August_2017.json', 'MA_September_2018.json', 'MA_November_2018.json', 'MA_April_2018.json', 'MA_October_2018.json', 'MA_May_2016.json', 'MA_February_2016.json', 'M

In [7]:
for cc in MA_COUNTRIES:
    print(cc)
    cc_path_str = re.sub(" ", "_", cc)
    br = MaBaserate(cc)
    cc_gsr = [x for x in gsr if x["Country"] == cc]
    print("There are {} GSR events".format(len(cc_gsr)))

    try:
        preds=br.make_predictions(WARN_START_DATE, WARN_END_DATE, history_delay=HISTORY_DELAY,
                                  gsr=cc_gsr)
        print(len(preds))

        pred_json = br.convert_warnings_to_json(preds)
    except AttributeError:
        pred_json = {"participant_id": "Mercury_Baserate", "payload": []}

    out_filename = "Baserate_MA_{0}_{1}_to_{2}.json".format(cc_path_str, 
                                                            WARN_START_DATE, WARN_END_DATE)
    out_path = os.path.join(MA_WARN_PATH, cc_path_str, out_filename)
    if True:
        with open(out_path, "w") as f:
            json.dump(pred_json, f, ensure_ascii=False, indent=2)
    print(out_path)

Iraq
There are 23163 GSR events
8471
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Iraq/Baserate_MA_Iraq_2018-01-09_to_2019-01-22.json
Syria
There are 71440 GSR events
23293
/Users/peterhaglich/Dropbox/Work/IARPA/Mercury/peterhaglich/mercury-challenge/data/baserate_warnings/baserate_ma_warnings/Syria/Baserate_MA_Syria_2018-01-09_to_2019-01-22.json


In [8]:
[p["Warning_ID"] for p in pred_json["payload"] if p["Latitude"] == "Latitude"]

[]

In [9]:
pred_json["payload"][0]

{'Event_Date': '2018-02-06',
 'timestamp': '2018-02-03T21:27:51.0',
 'Event_Subtype': 'Conflict',
 'Actor': 'Russian Military',
 'City': 'Jabal al Akrād',
 'Latitude': 35.65,
 'Longitude': 36.2,
 'State': 'Latakia',
 'Country': 'Syria',
 'Event_Type': 'Military Activity',
 'Warning_ID': 'Mercury_Baserate4355d390b28b41439190ac695698c78f'}